# Sesión 2: Walmart Project — Limpieza, Enriquecimiento y Resumen con KPIs

## Objetivos de la sesión
- **LIMPIAR**: normalizar nombres, estandarizar fechas por semana (`semana_limpia`).
- **ENRIQUECER**: unir `raw_ventas` con `stores` y `departments` vía `XLOOKUP/VLOOKUP`.
- **RESUMIR**: construir pivots para **KPI de eficiencia (ventas/m²)**, **participación** y **volatilidad (CV)**.
- Documentar supuestos y validaciones básicas del merge.


## 📅 Agenda 

* **Parte 1** — Limpieza de datos

* **Parte 2** — Enriquecimiento

* **Parte 3** — Resumen con KPIs vía tablas dinámicas



## Setup
- [walmart_project_data.xlsx]([/mnt/data/walmart_project_data.xlsx](https://docs.google.com/spreadsheets/d/1niRmx3GerZx5U8ZTBk1300oxfkjZW9i1/edit?usp=sharing&ouid=107698041790639854307&rtpof=true&sd=true)) — hojas: `raw_ventas`, `stores`, `departments`.


# Parte 1 — Limpieza de datos 

**Hoja de trabajo:** duplica `raw_ventas` → nómbrala `clean_ventas` (marca pestaña en **verde**).

### 0. Break rooms:

Identifica qué cambios harías a la hoja de cáculo.
-  Cambiarias nombres de columnas?
- Agregarías más columnas?
- Cambiarias algún valor de las columnas? Quizás los 0/1

### 1.1 Normaliza columnas
- Renombra a **snake_case** o nombres declarativos: `tienda | dept | fecha | ventas_semanales | es_feriado`
- Aplica:
  - `=TRIM()` para eliminar espacios sobrantes
  - `=LOWER()/UPPER()/PROPER()` para estandarizar texto
  - Validación de datos si corresponde (`es_feriado` ∈ {0,1})

### 1.2 Estandariza semana (clave temporal)
Crea columna **`semana_limpia`** (año-semana ISO; lunes como primer día):

- **Google Sheets**  
  `=YEAR(C2) & "-" & TEXT(WEEKNUM(C2, 2), "00")`

> Comprueba que todas las filas sean 2012. Si encuentras otras fechas, documenta el criterio (filtrar/ajustar).

**Salida esperada**: hoja `clean_ventas` lista para análisis:
`tienda | dept | fecha | ventas_semanales | es_feriado | semana_limpia`

---
# Parte 2 — Enriquecimiento (20–30 min)
**Objetivo**: añadir `tipo`, `tamaño`, `nombre_dept` a `clean_ventas` desde `stores` y `departments`.

### 2.1 Join por `Store` → `Type` y `Size`
En `clean_ventas`, inserta columnas:
- `tipo` (desde `stores[Type]`)
- `tamaño` (desde `stores[Size]`)

**Google Sheets — con `XLOOKUP`**  
`=XLOOKUP(A2, Stores!A:A, Stores!B:B, "NoMatch", 0))`

*(Sheets usa la misma sintaxis de XLOOKUP; con VLOOKUP, recuerda el índice de columna y `FALSE`)*

### 2.2 Join por `Dept` → `dept_name`
`=XLOOKUP(B2, Departments!A:A, Departments!B:B, "NoMatch", 0)` → `nombre_dept`

### 2.3 Verificación y troubleshooting
- Cuenta no coincidencias: `=COUNTIF(RangoResultados,"NoMatch")`
- Revisa duplicados en claves (`Store`, `Dept`) según tu diseño de unión.
- Documento “JOIN_DOC”:
  - Claves y cardinalidad asumidas
  - % de `NoMatch`
  - Decisiones tomadas ante errores (corrección, exclusión, imputación)

**Salida esperada**: `tienda | dept | fecha | ventas_semanales | es_feriado | semana_limpia | tipo | tamaño | nombre_dept`

---
# Parte 3 — Resumen con KPIs vía Tablas Dinámicas (20–30 min)

> Construye **una pivot por KPI** y deja los cálculos como **campos calculados** cuando aplique.

## KPI 1 — Eficiencia (Ventas por m²)
1) Pivot (por `tienda` o `nombre_dept`):  
   - Valores: **SUM(ventas_semanales)** y **AVERAGE(tamaño)**
2) Campo calculado **`ventasxmetro2`**:  
   `=SUM(ventas_semanales) / AVERAGE(tamaño)`
3) Ordena de mayor a menor.

**Salida**: columnas → `nombre_dept` | `SUM ventas` | `AVG tamaño` | `ventasxmetro2`

---

## KPI 2 — Participación del departamento (% sobre total)
1) Pivot por `nombre_dept` con **SUM(ventas_semanales)**  
2) Mostrar valor como **% del total**.

**Salida**: `nombre_dept` | `% del total de ventas`

---

## KPI 3 — Volatilidad (Coeficiente de Variación, CV)
1) Pivot por `nombre_dept` con:
   - `SUM(ventas_semanales)`
   - `STDEV(ventas_semanales)`
   - `AVERAGE(ventas_semanales)`
2) Campo calculado **`cv`**:  
   `=STDEV(ventas_semanales) / AVERAGE(ventas_semanales)`

**Interpretación**:  
- CV ~ 0 → estable; CV ≥ 0.5 → volátil; CV ≥ 1 → muy volátil.


---
## Entregables al cierre de la sesión (Parte 1 de 2)
- Hoja **`clean_ventas`** lista.
- Hoja **`JOIN_DOC`** con supuestos y verificaciones (conteo de `NoMatch`, duplicados).
- Tres pivots, uno por KPI (**eficiencia, participación, volatilidad**) con campos calculados.
- 2–3 bullets de hallazgos rápidos (sin dashboard todavía).

---
## Criterios de evaluación rápidos
- **Limpieza**: nomenclatura consistente y `semana_limpia` correcta.  
- **Enriquecimiento**: `XLOOKUP/VLOOKUP` correctos, verificación documentada.  
- **KPIs**: campos calculados y pivots claros, ordenados y legibles.  
- **Razonamiento**: Justificación de decisiones ante errores y outliers.


---
## Apéndice: fórmulas útiles y tips
- `=TRIM()`, `=CLEAN()`, `=PROPER()`, validación de datos (listas).  
- `=XLOOKUP(valor, rango_busqueda, rango_resultado, "NoMatch", 0)` (coincidencia exacta).  
- `=VLOOKUP(valor, tabla, índice, FALSE)` (evita coincidencia aproximada salvo que la necesites).  
- **Pivots**: usa “Mostrar valores como % del total” para participación.  
- **Documenta**: todo supuesto de join y tratamiento de `NoMatch` en `JOIN_DOC`.


## Cierre

- Kahoot
- Repaso de aprendizajes y próximos pasos

## Siguientes Pasos

- **Participación continua:** asistir a Co-Learning y a Sprint Focus, y usar los canales de Discord para hacer preguntas.
- **Recordatorios:** la grabación y recursos utilizados, se comparten al finalizar la sesión; en caso de necesitar apoyo adicional, agenda un 1:1.
